In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=400ee234813bb010b8e6b78796815375300b9f18ce4d85e430bce8d83537bd90
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
# Librerias
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# Carga de datos
from google.colab import drive
drive.mount("/content/drive")
file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore.csv"
reviews_file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore_user_reviews.csv"


Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
google_play_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(file_path, header=True, inferSchema=True)
user_reviews_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(reviews_file_path, header=True, inferSchema=True)
google_play_data = google_play_data.rdd
user_reviews_data = user_reviews_data.rdd
# Filtra las filas en las que la categoría no sea "1.9" y crea un nuevo RDD sin esa fila
google_play_data = google_play_data.filter(lambda x: x['Category'] != "1.9")


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# 27 - ¿Qué porcentaje de reviews contienen al nombre de la app en la review? (⭐)


Eliminar filas duplicadas basadas en la columna 'Translated_Review'

In [ ]:
unique_reviews_rdd = user_reviews_data.map(lambda x: (x['Translated_Review'], x)) \
    .distinct() \
    .map(lambda x: x[1])  # Para eliminar la clave temporal agregada por el map

Mapear cada review con su app como clave

In [ ]:
key_value_rdd = unique_reviews_rdd.map(lambda x: (x['App'], x))

Definir una función para verificar si el nombre de la aplicación está en la revisión

In [ ]:
def contains_app_name(app_name, review):
    return app_name.lower() in review.lower()

Filtrar las revisiones que contienen el nombre de la aplicación

In [ ]:
reviews_with_app_name_rdd = key_value_rdd.filter(lambda x: x[1]['Translated_Review'] is not None and contains_app_name(x[0], x[1]['Translated_Review']))

Contar el número total de revisiones en user_reviews_data

In [ ]:
total_reviews_count = unique_reviews_rdd.count()

Contar el número de revisiones que cumplen con la condición

In [ ]:
reviews_with_app_name_count = reviews_with_app_name_rdd.count()

Calcular el porcentaje

In [ ]:
percentage = (reviews_with_app_name_count / total_reviews_count) * 100

In [ ]:
percentage

2.0307050425372406